# tabs

> Tab switching route handlers

In [ ]:
#| default_exp routes.tabs

In [ ]:
#| export
from typing import Tuple, Dict, Callable

from fasthtml.common import APIRouter

from cjm_fasthtml_interactions.core.state_store import get_session_id

from cjm_transcript_source_select.models import SelectionUrls
from cjm_transcript_source_select.routes.core import (
    WorkflowStateStore, _get_step_state, _update_step_state
)
from cjm_transcript_source_select.routes.local_files import (
    _get_local_files_provider, _get_local_files_config
)
from cjm_transcript_source_select.components.source_browser import (
    _render_source_browser
)
from cjm_transcript_source_select.components.local_files import (
    _render_local_files_browser, _get_file_browser_state
)
from cjm_transcript_source_select.components.step_renderer import (
    _render_tab_headers
)
from cjm_transcript_source_select.services.source import SourceService
from cjm_transcript_source_select.services.source_utils import calculate_next_tab

## Tab Switch Handler

In [ ]:
#| export
def _handle_tab_switch(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for queries
    request,  # FastHTML request object
    sess,  # FastHTML session object
    direction: str,  # Direction: "prev", "next", "db", or "files"
    urls: SelectionUrls,  # URL bundle for rendering
):  # Tuple of inner content and OOB tab headers
    """Switch between Plugin DB and Local Files tabs."""
    session_id = get_session_id(sess)
    
    # Get current state
    step_state = _get_step_state(state_store, workflow_id, session_id)
    selected_sources = step_state.get("selected_sources", [])
    grouping_mode = step_state.get("grouping_mode", "media_path")
    external_db_paths = step_state.get("external_db_paths", [])
    
    # Determine new tab
    workflow_state = state_store.get_state(workflow_id, session_id)
    current_tab = workflow_state.get("source_tab", "db")
    new_tab = calculate_next_tab(direction, current_tab, ["db", "files"])
    
    # Save new tab state
    workflow_state["source_tab"] = new_tab
    state_store.update_state(workflow_id, session_id, workflow_state)
    
    # Render the content for the new active tab
    if new_tab == "db":
        # Get transcriptions for source browser
        all_transcriptions = source_service.query_transcriptions(limit=500)
        sources = source_service.get_available_sources()
        
        content = _render_source_browser(
            transcriptions=all_transcriptions,
            sources=sources,
            selected_sources=selected_sources,
            add_url=urls.add,
            remove_url=urls.remove,
            preview_url=urls.preview,
            select_all_url=urls.select_all,
            filter_url=urls.filter,
            grouping_mode=grouping_mode,
            grouping_change_url=urls.grouping_change,
        )
    else:
        # Render local files browser content with file browser library
        provider = _get_local_files_provider()
        config = _get_local_files_config()
        browser_state = _get_file_browser_state(step_state, provider.get_home_path())
        
        # Sync browser selection state with external_db_paths for checkbox display
        browser_state.selection.selected_paths = list(external_db_paths)
        _update_step_state(state_store, workflow_id, session_id, file_browser_state=browser_state.to_dict())
        
        content = _render_local_files_browser(
            browser_state=browser_state,
            external_paths=external_db_paths,
            provider=provider,
            config=config,
            navigate_url=urls.browse_directory,
            select_url=urls.add_external,
            remove_url=urls.remove_external,
            home_path=provider.get_home_path(),
        )
    
    # Render tab headers with OOB swap to update checked state
    tab_headers_oob = _render_tab_headers(
        active_tab=new_tab,
        tab_switch_url=urls.tab_switch,
        oob=True,
    )
    
    return (content, tab_headers_oob)

## Router

In [ ]:
#| export
def init_tabs_router(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for queries
    prefix: str,  # Route prefix (e.g., "/workflow/selection/tabs")
    urls: SelectionUrls,  # URL bundle for rendering
) -> Tuple[APIRouter, Dict[str, Callable]]:  # (router, route_dict)
    """Initialize tab switching routes."""
    router = APIRouter(prefix=prefix)

    @router
    def tab_switch(request, sess, direction: str):
        """Switch between source tabs."""
        return _handle_tab_switch(
            state_store, workflow_id, source_service,
            request, sess, direction, urls=urls,
        )

    routes = {
        "tab_switch": tab_switch,
    }

    return router, routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()